In [2]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.5 MB/s eta 0:00:00
   

In [61]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = ''

In [62]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning, module='jupyter_client')

In [64]:
question = "What kinds of pets do I like?"
document = "My favorite pet is a cat."

In [65]:
import tiktoken

In [66]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(question, "cl100k_base")

8

In [67]:
!pip install langchain-google-genai

In [25]:
os.environ['GOOGLE_API_KEY'] = ''

In [26]:
import bs4
import langchainhub as hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings


In [27]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embd = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

In [28]:
import time
from google.api_core import exceptions as google_exceptions
from langchain_google_genai import GoogleGenerativeAIEmbeddings

def embed_with_retry(embedding_model, text, retries=3):
    for attempt in range(retries):
        try:
            return embedding_model.embed_query(text)
        except Exception as e:
            # Check if it is a Resource Exhausted error (429)
            if "429" in str(e) or "RESOURCE_EXHAUSTED" in str(e):
                wait_time = 60  # Wait 60 seconds (safe buffer above the 54s suggested)
                print(f"Quota hit. Waiting {wait_time} seconds before retry {attempt + 1}/{retries}...")
                time.sleep(wait_time)
            else:
                raise e # Raise other errors immediately
    raise Exception("Max retries exceeded for embedding API.")

In [29]:
query_result = embed_with_retry(embd,question)

In [30]:
document_result = embd.embed_query(document)


In [31]:
print(f"query_len=",len(query_result)," doc_len=",len(document_result))

query_len= 768  doc_len= 768


But how does this embedding thing relly work? what is the MATH behind it?

Find that in my work repo

In [52]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, document_result)
print("Cosine Similarity:", similarity)

Cosine Similarity: 0.6699860094539072


In [53]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()


This code uses SoupStrainer to selectively extract only the core content (title and body) while ignoring "noise" like navigation links or footers. This ensures the MiniTransformer focuses its attention on high-quality information without wasting its limited context window on irrelevant text.

In [54]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50)

splits = text_splitter.split_documents(blog_docs)
len(splits)

50

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough. The default list is ["\n\n", "\n", " ", ""].

The code above breaks long documents into smaller, 300-token "chunks" with a 50-token overlap between them to preserve context.

In [55]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma

# Create the vectorstore
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embd
)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3}) # Retrieve top 3 chunks

In [57]:
docs = retriever.invoke("What is Task Decomposition?")

In [60]:
len(docs)

3

In [69]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'), additional_kwargs={})])

In [74]:
# LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
# Chain
chain = prompt | llm


In [76]:
message =chain.invoke({"context":docs,"question":"What is Task Decomposition?"})

In [78]:
message.content

'Task decomposition is a technique used to break down complex tasks into smaller, simpler, and more manageable steps. It enhances model performance on complex tasks by instructing the model to "think step by step."\n\nThis process can be achieved in several ways:\n1.  **By LLM with simple prompting:** Using prompts like "Steps for XYZ." or "What are the subgoals for achieving XYZ?".\n2.  **By using task-specific instructions:** For example, "Write a story outline." for writing a novel.\n3.  **With human inputs.**\n\nChain of Thought (CoT) is a standard prompting technique that utilizes task decomposition, and Tree of Thoughts extends CoT by exploring multiple reasoning possibilities at each step, creating a tree structure of thought steps.'

In [91]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

'Task decomposition is a technique used to break down complex tasks into smaller, simpler, and more manageable steps. It enhances model performance on complex tasks by instructing the model to "think step by step."\n\nThis process can be achieved in several ways:\n1.  **By LLM with simple prompting:** Using prompts like "Steps for XYZ." or "What are the subgoals for achieving XYZ?".\n2.  **By using task-specific instructions:** For example, "Write a story outline." for writing a novel.\n3.  **With human inputs.**\n\nChain of Thought (CoT) is a standard prompting technique that utilizes task decomposition, and Tree of Thoughts extends CoT by exploring multiple reasoning possibilities at each step, creating a tree structure of thought steps.'